In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
from scipy.stats import chi2_contingency
from scipy.stats import fisher_exact
from scipy.stats import kruskal, mannwhitneyu

In [2]:
data = pd.read_csv(r"C:\Users\asus\Desktop\Personas\Clustered_df.csv")
data = data.drop(data.columns[0], axis=1)
print(data.columns)

Index(['Age', 'Education', 'Affective Symptoms', 'Rumination',
       'Behavioural Symptoms', 'Anxiety Personal Impact',
       'Attribution Skepticism', 'Impact Skepticism', 'Trend Skepticism',
       'Response Skepticism', 'Male', 'Female', 'Non-binary', 'Single',
       'Married', 'Divorced', 'Widowed', 'Separated', 'Income', 'Cluster'],
      dtype='object')


# Statistical Analysis for attributes comparison 

## CATEGORICAL VARIABLES (Chi-Squared and Fisher's Exact Test)

In [3]:
df_categorical = data
columns_to_drop = ['Age', 'Income']
df_categorical = df_categorical.drop(columns=columns_to_drop)

print(df_categorical.columns)

Index(['Education', 'Affective Symptoms', 'Rumination', 'Behavioural Symptoms',
       'Anxiety Personal Impact', 'Attribution Skepticism',
       'Impact Skepticism', 'Trend Skepticism', 'Response Skepticism', 'Male',
       'Female', 'Non-binary', 'Single', 'Married', 'Divorced', 'Widowed',
       'Separated', 'Cluster'],
      dtype='object')


### Computation of p-values for each feature, comparing the three groups together (1 test per feature)

In [4]:
cluster_colonna = 'Cluster'

# Itera su tutte le colonne tranne la colonna 'Cluster'
for attributo_selezionato in df_categorical.columns[df_categorical.columns != cluster_colonna]:
    # Calcola la tabella di contingenza per l'attributo corrente
    contingency_table = pd.crosstab(df_categorical[attributo_selezionato], df_categorical[cluster_colonna])
    
    # Stampa il risultato
    print(f"\nTabella di contingenza per l'attributo {attributo_selezionato}:\n{contingency_table}")
    
    all_values_above_5 = np.all(contingency_table >= 5)

    if all_values_above_5:  # Apply Chi-squared
        print(f"\nFor the feature {attributo_selezionato}, all expected frequencies are at least 5. I can perform Chi-Squared test")
        # Perform Chi-squared test
        chi2_stat, p_value, _, _ = chi2_contingency(contingency_table)

        # Results:
        print(f"Feature: {attributo_selezionato}")
        print(f"Chi-square Statistic: {chi2_stat}")
        print(f"P-Value: {p_value}")
        
    else: 
        print(f"\nFor the feature {attributo_selezionato}, some expected frequencies are below 5. Consider alternative methods.")

        print("I have to perform Fisher's Exact Test for non binary features.")

        # I performed Fisher's Exact Test for non binary features in R and i got the following results:
        # See the "R code (groups together).r"
        results_fisher_together = [
            {"Test": "Education", "p_value": 0.0004997501},
            {"Test": "Affective Symptoms", "p_value": 0.0004997501},
            {"Test": "Rumination", "p_value": 0.0004997501},
            {"Test": "Behavioural Symptoms", "p_value": 0.0004997501},
            {"Test": "Anxiety Personal Impact", "p_value": 0.0004997501},
            {"Test": "Attribution Skepticism", "p_value": 0.0004997501},
            {"Test": "Impact Skepticism", "p_value": 0.0004997501},
            {"Test": "Trend Skepticism", "p_value": 0.0004997501},
            {"Test": "Response Skepticism", "p_value": 0.9375312},
            {"Test": "Single", "p_value": 0.0004997501},
            {"Test": "Widowed", "p_value": 0.1454273},
            {"Test": "Separated", "p_value": 0.0004997501},
            ]

         # Create a DataFrame with named rows
        results_fisher_together_df = pd.DataFrame(results_fisher_together)
        results_fisher_together_df.set_index("Test", inplace=True)
        
        feature_val = str(attributo_selezionato)

        # Extract the p-value for the specified feature
        p_value_together = results_fisher_together_df.loc[feature_val, 'p_value']

        print(f"Feature: {attributo_selezionato}")
        print(f"Fisher P value: {p_value_together}")

            


Tabella di contingenza per l'attributo Education:
Cluster     1   2   3
Education            
5.000000    0   8   0
8.000000    0  24  12
13.000000  11  14  31
14.100629   1   0   0
18.000000  18   7   9
22.000000  15   0   2
25.000000   8   0   0

For the feature Education, some expected frequencies are below 5. Consider alternative methods.
I have to perform Fisher's Exact Test for non binary features.
Feature: Education
Fisher P value: 0.0004997501

Tabella di contingenza per l'attributo Affective Symptoms:
Cluster              1   2   3
Affective Symptoms            
0.000000             3   0   0
0.250000             4   0   0
0.500000             8   0   0
0.750000            11   0   1
1.000000             5   0   4
1.250000             6   0   5
1.333333             0   0   1
1.500000             5   0   6
1.666667             1   0   0
1.750000             5   1   8
2.000000             3   0  10
2.250000             1   5   6
2.500000             0  15   1
2.666667          

### Computation of p-values for each feature, comparing the three groups in pairs (3 test per feature)

In [5]:
# List of all categorical variables in the DataFrame
categorical_variables = df_categorical.columns.tolist()
cluster_colonna = 'Cluster'

# Significance level after Bonferroni correction
alpha = 0.05
alpha_adjusted = alpha / 3
count_dict = {variable: 0 for variable in categorical_variables}

for feature in df_categorical.columns[df_categorical.columns != cluster_colonna]:
    # Create a contingency table
    contingency_table = pd.crosstab(df_categorical[feature], df_categorical[cluster_colonna])

    # Iterate over all possible pairs of clusters
    clusters = df_categorical[cluster_colonna].unique()
    for i in range(len(clusters)):
        for j in range(i + 1, len(clusters)):
            # Create a contingency table for the pair of clusters
            subset_table = contingency_table[[clusters[i], clusters[j]]]
            print (f"\n {subset_table}")
            # Check if all expected frequencies are above 5
            all_values_above_5 = np.all(subset_table >= 5)

            if all_values_above_5:  # Apply Chi-square
                print(f"\nFor the feature {feature}, all expected frequencies are at least 5. I can perform Chi-Squared test")
                # Perform Chi-square test
                chi2_stat, p_value, _, _ = chi2_contingency(subset_table)

                # Results:
                print(f"Feature: {feature}, Cluster Pair: {clusters[i]} vs {clusters[j]}")
                print(f"Chi-square Statistic: {chi2_stat}")
                print(f"P-Value: {p_value}")
                print(f"Adjusted alpha with Bonferroni: {alpha_adjusted}")

                # Check if the result is statistically significant after Bonferroni correction
                if p_value < alpha_adjusted:
                    print(f"The clusters {clusters[i]} and {clusters[j]} are statistically different regarding the feature {feature}.")
                else:
                    print(f"The clusters {clusters[i]} and {clusters[j]} are NOT statistically different regarding the feature {feature}.")
                    count_dict[feature] += 1


            else: 
                print(f"\nFor the feature {feature}, some expected frequencies are below 5. Consider alternative methods.")
                
                if df_categorical[feature].nunique() < 3: # Apply Fisher
                    print("I can perform Fisher's Exact Test.")
                    # Perform Fisher’s Exact Test
                    odds_ratio, p_value = fisher_exact(subset_table)

                    # Results:
                    print(f"Feature: {feature}")
                    print(f"Fisher Odds Ratio: {odds_ratio}, p-value: {p_value}")
                    print(f"Adjusted alpha with Bonferroni: {alpha_adjusted}")

                    # Check if the result is statistically significant after Bonferroni correction
                    if p_value < alpha_adjusted:
                        print(f"The clusters {clusters[i]} and {clusters[j]} are statistically different regarding the feature {feature}.")
                    else:
                        print(f"The clusters {clusters[i]} and {clusters[j]} are NOT statistically different regarding the feature {feature}.")
                        count_dict[feature] += 1


                else:
                    print("I have to perform Fisher's Exact Test for non binary features.")

                    # I performed Fisher's Exact Test for non binary features in R and i got the following results:
                    # See the "R code (groups paired).r"
                    results_fisher = [
                        {"Test": "Education", "i":"2", "j":"1", "p_value": 0.0004997501},
                        {"Test": "Education", "i":"2", "j":"3","p_value": 0.0004997501},
                        {"Test": "Education", "i":"1", "j":"3","p_value": 0.0004997501},
                        {"Test": "Affective Symptoms", "i":"2", "j":"1","p_value": 0.0004997501},
                        {"Test": "Affective Symptoms", "i":"2", "j":"3","p_value": 0.0004997501},
                        {"Test": "Affective Symptoms", "i":"1", "j":"3","p_value": 0.0004997501},
                        {"Test": "Rumination", "i":"2", "j":"1","p_value": 0.0004997501},
                        {"Test": "Rumination", "i":"2", "j":"3","p_value": 0.0004997501},
                        {"Test": "Rumination", "i":"1", "j":"3","p_value": 0.0004997501},
                        {"Test": "Behavioural Symptoms", "i":"2", "j":"1","p_value": 0.0004997501},
                        {"Test": "Behavioural Symptoms", "i":"2", "j":"3","p_value": 0.0004997501},
                        {"Test": "Behavioural Symptoms", "i":"1", "j":"3","p_value": 0.0004997501},
                        {"Test": "Anxiety Personal Impact", "i":"2", "j":"1","p_value": 0.0004997501},
                        {"Test": "Anxiety Personal Impact", "i":"2", "j":"3","p_value": 0.0004997501},
                        {"Test": "Anxiety Personal Impact", "i":"1", "j":"3","p_value": 0.0004997501},
                        {"Test": "Attribution Skepticism", "i":"2", "j":"1","p_value": 0.0004997501},
                        {"Test": "Attribution Skepticism", "i":"2", "j":"3","p_value": 0.0004997501},
                        {"Test": "Attribution Skepticism", "i":"1", "j":"3","p_value": 0.0004997501},
                        {"Test": "Impact Skepticism", "i":"2", "j":"1","p_value": 0.0004997501},
                        {"Test": "Impact Skepticism", "i":"2", "j":"3","p_value": 0.0004997501},
                        {"Test": "Impact Skepticism", "i":"1", "j":"3","p_value": 0.2618691},
                        {"Test": "Trend Skepticism", "i":"2", "j":"1","p_value": 0.0004997501},
                        {"Test": "Trend Skepticism", "i":"2", "j":"3","p_value": 0.0004997501},
                        {"Test": "Trend Skepticism", "i":"1", "j":"3","p_value": 0.0004997501},
                        {"Test": "Response Skepticism", "i":"2", "j":"1","p_value": 0.6311844},
                        {"Test": "Response Skepticism", "i":"2", "j":"3","p_value": 0.934033},
                        {"Test": "Response Skepticism", "i":"1", "j":"3","p_value": 0.941029}
                    ]

                    # Create a DataFrame with named rows
                    results_fisher_df = pd.DataFrame(results_fisher)
                    results_fisher_df.set_index("Test", inplace=True)
                    
                    i_value = str(clusters[i])
                    j_value = str(clusters[j])
                    feature_value = str(feature)

                    # Filter the DataFrame based on conditions
                    filtered_df = results_fisher_df[(results_fisher_df['i'] == i_value) & (results_fisher_df['j'] == j_value)]
                    # Extract the p-value for the specified feature
                    p_value_to_compare = filtered_df.loc[feature_value, 'p_value']

                    print(f"Feature: {feature}")
                    print(f"Fisher P value: {p_value_to_compare}")
                    print(f"Adjusted alpha with Bonferroni: {alpha_adjusted}")

                    if p_value_to_compare < alpha_adjusted:
                        print(f"The clusters {clusters[i]} and {clusters[j]} are statistically different regarding the feature {feature}.")
                    else:
                        print(f"The clusters {clusters[i]} and {clusters[j]} are NOT statistically different regarding the feature {feature}.")
                        count_dict[feature] += 1




 Cluster     2   1
Education        
5.000000    8   0
8.000000   24   0
13.000000  14  11
14.100629   0   1
18.000000   7  18
22.000000   0  15
25.000000   0   8

For the feature Education, some expected frequencies are below 5. Consider alternative methods.
I have to perform Fisher's Exact Test for non binary features.
Feature: Education
Fisher P value: 0.0004997501
Adjusted alpha with Bonferroni: 0.016666666666666666
The clusters 2 and 1 are statistically different regarding the feature Education.

 Cluster     2   3
Education        
5.000000    8   0
8.000000   24  12
13.000000  14  31
14.100629   0   0
18.000000   7   9
22.000000   0   2
25.000000   0   0

For the feature Education, some expected frequencies are below 5. Consider alternative methods.
I have to perform Fisher's Exact Test for non binary features.
Feature: Education
Fisher P value: 0.0004997501
Adjusted alpha with Bonferroni: 0.016666666666666666
The clusters 2 and 3 are statistically different regarding the featu

In [6]:
for variable, count in count_dict.items():
    print(f"Number of p-values > 0.0167 for {variable}: {count}")

Number of p-values > 0.0167 for Education: 0
Number of p-values > 0.0167 for Affective Symptoms: 0
Number of p-values > 0.0167 for Rumination: 0
Number of p-values > 0.0167 for Behavioural Symptoms: 0
Number of p-values > 0.0167 for Anxiety Personal Impact: 0
Number of p-values > 0.0167 for Attribution Skepticism: 0
Number of p-values > 0.0167 for Impact Skepticism: 1
Number of p-values > 0.0167 for Trend Skepticism: 0
Number of p-values > 0.0167 for Response Skepticism: 3
Number of p-values > 0.0167 for Male: 3
Number of p-values > 0.0167 for Female: 3
Number of p-values > 0.0167 for Non-binary: 3
Number of p-values > 0.0167 for Single: 1
Number of p-values > 0.0167 for Married: 1
Number of p-values > 0.0167 for Divorced: 3
Number of p-values > 0.0167 for Widowed: 3
Number of p-values > 0.0167 for Separated: 1
Number of p-values > 0.0167 for Cluster: 0


##  NUMERICAL VARIABLES (Kruskal-Wallis Test + Pairwise MannWhitney U Tests)

In [7]:
df_numerical = data
columns_to_drop = ['Education', 'Affective Symptoms', 'Rumination', 'Behavioural Symptoms','Anxiety Personal Impact', 'Attribution Skepticism', 'Impact Skepticism', 'Trend Skepticism', 'Response Skepticism', 'Male',
       'Female', 'Non-binary', 'Single', 'Married', 'Divorced', 'Widowed',
       'Separated']
#columns_to_drop = ['Cluster']
df_numerical = df_numerical.drop(columns=columns_to_drop)
print(df_numerical.columns)

Index(['Age', 'Income', 'Cluster'], dtype='object')


In [8]:
alpha_corrected = 0.05 / 3
# Calculate the Kruskal-Wallis test for each column with respect to the groups defined by 'Clusters'.
columns_to_test = ['Age', 'Income']
for column in columns_to_test:
    groups = [df_numerical[column][df_numerical['Cluster'] == i] for i in range(1, 4)]
    kruskal_stat, kruskal_p_value = kruskal(*groups)
    print(f"Kruskal-Wallis Test for {column}: Statistic = {kruskal_stat}, p-value = {kruskal_p_value}")

    # If the p-value of the Kruskal-Wallis test is significant, perform Mann-Whitney U pairwise tests
    if kruskal_p_value < 0.05:
        pairwise_combs = list(combinations(range(1, 4), 2))
        for comb in pairwise_combs:
            group1 = groups[comb[0] - 1]
            group2 = groups[comb[1] - 1]
            mannwhitneyu_stat, mannwhitneyu_p_value = mannwhitneyu(group1, group2)
            # Comparison with Bonferroni's corrected threshold
            if mannwhitneyu_p_value < alpha_corrected:
                print(f"Mann-Whitney U Test between cluster {comb[0]} and cluster {comb[1]} for {column}: "
                      f"Statistic = {mannwhitneyu_stat}, p-value = {mannwhitneyu_p_value} (Statistical difference)")
            else:
                print(f"Mann-Whitney U Test between cluster {comb[0]} and cluster {comb[1]} for {column}: "
                      f"Statistic = {mannwhitneyu_stat}, p-value = {mannwhitneyu_p_value} (No statistical difference)")

Kruskal-Wallis Test for Age: Statistic = 104.89273324283666, p-value = 1.6704457674798354e-23
Mann-Whitney U Test between cluster 1 and cluster 2 for Age: Statistic = 7.0, p-value = 1.01518886253871e-18 (Statistical difference)
Mann-Whitney U Test between cluster 1 and cluster 3 for Age: Statistic = 10.0, p-value = 8.07241154519579e-19 (Statistical difference)
Mann-Whitney U Test between cluster 2 and cluster 3 for Age: Statistic = 1560.5, p-value = 0.42033942889913023 (No statistical difference)
Kruskal-Wallis Test for Income: Statistic = 84.69148565333114, p-value = 4.0689050762271256e-19
Mann-Whitney U Test between cluster 1 and cluster 2 for Income: Statistic = 2652.5, p-value = 3.0953455055822063e-15 (Statistical difference)
Mann-Whitney U Test between cluster 1 and cluster 3 for Income: Statistic = 1312.0, p-value = 0.45951130427912523 (No statistical difference)
Mann-Whitney U Test between cluster 2 and cluster 3 for Income: Statistic = 146.5, p-value = 1.1825570172755493e-15 (S